# This notebook shows how to steady-state enzyme kinetics using non-linear fit and linearization methods 


#### Import reqired python modules

In [67]:
import numpy as np # numeric python module for vectorized operations on arrays 
import pandas # convenience wrap for numpy 
from scipy.integrate import odeint # function for solving ordinary differential equations
from scipy.special import lambertw # implementation of Lambert W function
from scipy.optimize import minimize,basinhopping,differential_evolution #optimization algorithms
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt # plotting graphics library
%matplotlib auto
plt.ioff() # I prefer to output graphs into separate windows rather than showing them inline.
#%matplotlib inline 

Using matplotlib backend: Qt5Agg


# Set up main variables

In [75]:
#linear scale of [S]o
listSo=  [0,20,40,60,80,100,120,140,160] # initial substrate  concentrations (list if floats)
Smax=160
#reciprocal scale of [S]o
#listSo=  [Smax/7,Smax/6,Smax/5,Smax/4,Smax/3,Smax/2,Smax/1] # initial substrate  concentrations (list if floats)
listS_STR = list(map(str, listSo))
listI=  [0,20,40,60,80,100,120,140,160]
listI_STR = list(map(str, listI))

noise_level=.001 # we may add some random noise to the data
noise=False #if False, no noise will be added to original data 
time_cutoff=2 # number of data points used for linear regression to estimate [S]o 

simulate_data=True #if False, data will be loaded from '../data/simulatedMM_variable_S.csv'  # Input experimental data
simulate_rates=True #if False, data will be loaded from '../data/rates.csv'
KM=40
Vm=4
KI=5
exact='True MM parameters: KM={}; Vmax={}; KI={}'.format(KM,Vm,KI)


## Generate a data set for competitive inhibition using steady-state approximation;

$$EI \underset{k_{3}}
{\overset{k_{-3}}{\rightleftharpoons}} E+S+I \underset{k_{-1}}
{\overset{k_{1}}{\rightleftharpoons}} ES\overset{k_{2}}{\rightarrow}E+P$$
$$-\frac{\mathrm{d}S }{\mathrm{d} t}=\frac{V_{max}[S]}{{K_{M}}\left( 1+\frac{[I]}{K_{I}}\right )+[S]}
$$

In [76]:
def model_comp_inh(S,t,*args): 
    '''Standard Quasi Steady-State Approximation(Michaelis-Menten equation)'''
    KM,KI,Vm=args #These are parameters that have to be optimized
    dSdt=-Vm*S/(KM*(1+I/KI)+S) # differential equation
    return (dSdt)# returns rates of concentration change
t = np.linspace(0,99,100) # make an array of time points
velocities=pandas.DataFrame()
velocities['S']=listSo
for I in listI:
    data=pandas.DataFrame() #initiate a Pandas DataFrame for data collection
    data['time']=t #make first column containing independent variable time
    #I=100
    for S in listSo:
        args=(KM,KI,Vm)
        z = odeint(model_comp_inh,S,t,args)
        z=z[:,0]
        if noise==True:
            noiseZ = np.random.normal(0, 1, z.shape)*z.max()*noise_level
            z=z+noiseZ
        data[str(S)]=z
    #print(data)
    data=data[:time_cutoff]
    V=[]
    for conc in listS_STR:
        DF=-np.polyfit(data.time,data[str(conc)],1)
        V.append(DF[0])
    velocities[str(I)]=V
#for col in velocities.columns:
#    velocities['1/'+col]=1/velocities[col]
#velocities=velocities.replace([np.inf, -np.inf], np.nan)
#velocities.dropna(inplace=True)

#velocities=velocities.drop('S', axis=1)
velocities.plot(x='S',y=listI_STR)
#velocities
plt.show()

#### Calculate apparent KM and apparent Vmax using non-linear fitting 

In [77]:
def model(S,KM,Vmax):
    return Vmax*S/(KM+S)
apparentMM=pandas.DataFrame()
apparentMM['I']=listI
aKM=[]
aVmax=[]
for col in velocities.columns:               
    if not col=='S':
        popt,pcov = curve_fit(model, velocities.S, velocities[col])
        aKM.append(popt[0])
        aVmax.append(popt[1])
apparentMM['aKM']=aKM
apparentMM['aVmax']=aVmax

apparentMM.plot(x='I',y='aKM',kind='scatter')
plt.show()
apparentMM

I          aKM     aVmax
0    0    41.579355  4.026509
1   20   202.741474  4.018290
2   40   363.138270  4.013509
3   60   523.343673  4.010669
4   80   683.470393  4.008810
5  100   843.553975  4.007490
6  120  1003.611154  4.006502
7  140  1163.656309  4.005749
8  160  1323.693429  4.005157

Since 
$$K_{M,apparent}=\frac{K_{M}[I]}{K_{I}}+{K_{M}}$$
KI can be calculated from parameters of linear regression as
$$K_{I}=\frac{b}{a}$$
using covariance matrix and law of propagation of uncertainty for ratios we obtain std for Km as
$$\sigma =\frac{b}{a}\sqrt{\frac{\sigma _{a}^{2}}{a^{2}}+\frac{\sigma _{a}^{2}}{a^{2}}-\frac{\sigma _{ab}}{ab}}$$

In [78]:
PF,cov=np.polyfit(apparentMM['I'],apparentMM['aKM'],1,cov=True)
a=PF[0]
b=PF[1]
calcKI=b/a
calcKIstd=calcKI*np.sqrt(cov[0,0]/a**2+cov[1,1]/b**2-cov[0,1]/a/b)
print('Calculated KI={}+/-{}'.format(calcKI,calcKIstd))
print(exact)

Calculated KI=5.290050612677238+/-0.03746402285774003
True MM parameters: KM=40; Vmax=4; KI=5
